In [1]:
import numpy as np
import time
import pickle
import os
import matplotlib.pyplot as plt
import warnings
import pandas as pd

warnings.filterwarnings('ignore')

In [2]:
print("----------------Reading the Data-------------------------")
PATH = os.getcwd()
os.chdir('Alphabets/')

X_train = pd.read_csv('train.csv', sep=',', header=None, index_col=False)
X_test = pd.read_csv('test.csv', sep=',', header=None, index_col=False)
np.random.shuffle(X_train.to_numpy())
train_class = X_train[X_train.columns[-1]]
test_actual_class = X_test[X_test.columns[-1]]

X_train = X_train.drop(X_train.columns[-1], axis=1)
X_test = X_test.drop(X_test.columns[-1], axis=1)

print("----------------Data Reading completed-------------------")

os.chdir('../')

X_train = X_train/255
X_test = X_test/255

m = X_train.shape[0] # Number of Training Samples

X_valid = X_train.iloc[(int(0.85*m)):]
valid_class = train_class[(int(0.85*m)):]
X_train = X_train.iloc[0:int(0.85*m)]
train_class = train_class[0:int(0.85*m)]


m = X_train.shape[0] # Number of Training Samples
n = X_train.shape[1] # Number of input features

print("The total number of training samples = {}".format(m))
print("The total number of validation samples = {}".format(X_valid.shape[0]))

print("The number of features = {}".format(n))

----------------Reading the Data-------------------------
----------------Data Reading completed-------------------
The total number of training samples = 11050
The total number of validation samples = 1950
The number of features = 784


In [3]:
#To get the one hot encoding of each label
print("--------Perform 1-hot encoding of class labels------------")

train_class_enc = pd.get_dummies(train_class).to_numpy()
valid_class_enc = pd.get_dummies(valid_class).to_numpy()
test_actual_class_enc = pd.get_dummies(test_actual_class).to_numpy()

--------Perform 1-hot encoding of class labels------------


In [4]:
#Add the intercept term to the data samples both in training and test dataset
X_train = np.hstack((np.ones((m,1)),X_train.to_numpy()))
X_valid = np.hstack((np.ones((X_valid.shape[0],1)), X_valid.to_numpy()))
X_test = np.hstack((np.ones((X_test.shape[0],1)),X_test.to_numpy()))

In [5]:
lr = 0.1
arch_test = [1,5,10,50,100]
arch = [arch_test[3]] #means one hidden layer with 2 perceptrons 
batch_size = 100 # Mini-Batch Size
r = np.max(train_class) + 1 # Default value of the number of classes = 26

In [6]:
#Mini-Batch formation
mini_batch = [(X_train[i:i+batch_size,:], train_class_enc[i:i+batch_size]) for i in range(0, m, batch_size)]
print("The number of mini-batches formed is = {}".format(len(mini_batch)))

The number of mini-batches formed is = 111


In [7]:
#Theta Initialization 
def theta_init(arch=[50]):
    theta = []
    for i in range(len(arch)+1):
        if i == 0:
            dim0=n+1
            dim1=arch[i]
        elif (i == len(arch)):
            dim0=arch[i-1]
            dim1 = r
        else:
            dim0=arch[i-1]
            dim1= arch[i]

        theta.append(2*np.random.random((dim0, dim1))-1)
        #theta.append(np.zeros((dim0, dim1)))
    return theta

In [8]:
def activation(x):
    return 1/(1+np.exp(-x))

In [9]:
def forward_prop(data, theta):
    fm = []
    fm.append(data)
    for l in range(len(theta)):
        fm.append(activation(np.dot(fm[l], theta[l])))
    return fm

In [10]:
def cost_total(X, theta, Y, m):
    fm = forward_prop(X, theta)
    cost = (1/(2*m))*np.sum((Y-fm[-1])**2)
    return cost

In [11]:
def calc_accuracy(data, theta, actual_class):
    pred_class = forward_prop(data, theta)
    test_pred_class = pred_class[-1]
    for i in range(len(test_pred_class)):
        test_pred_class[i][test_pred_class[i] == np.max(test_pred_class[i])] = 1
        test_pred_class[i][test_pred_class[i] != np.max(test_pred_class[i])] = 0


    test_acc = 0
    for i in range(len(actual_class)):
        if (np.array_equal(test_pred_class[i], actual_class[i])):
            test_acc+=1
    test_acc /= data.shape[0]

    #print("The Test Accuracy of the model = {}%".format(test_acc*100))
    return (test_acc*100)

In [12]:
epochs = []
train_accuracy = []
test_accuracy = []
valid_accuracy=[]
train_time = []

In [17]:
lr0=1.5
arch_test=[50]
for i in range(len(arch_test)):
    theta = theta_init([arch_test[i]])
    fm = forward_prop(X_train, theta)
    ite=1
    epoch = 0
    start = time.time()
    cost_init = cost_total(X_valid, theta, valid_class_enc, m)

    while(True):
        count = 0

        print("Initial Cost on Val dataset for this epoch {} = {}".format(epoch, cost_init))

        for b in mini_batch:
            lr = lr0/ite
            if (lr < 0.1): lr = 0.1
            #print("learning rate = ", lr)
            X_b = b[0]
            Y_b = b[1]
            fm = forward_prop(X_b, theta)
            delta = [None]*len(fm)

            if (count % 60 == 0):
                print("Error on this batch = "+str(cost_total(X_b, theta, Y_b, batch_size)))
            #Backward Propagation

            for l in range(len(fm)-1, 0, -1):
                if (l == len(fm)-1):
                    delta[l] = ((1/batch_size)*(Y_b - fm[l])*fm[l]*(1-fm[l]))
                else:
                    delta[l]=(np.dot(delta[l+1], theta[l].T)*fm[l]*(1-fm[l]))

            #delta_l1 = np.dot(delta_l2, theta2.T)*l1*(1-l1)

            for t in range(len(theta)):
                theta[t] += lr*np.dot(fm[t].T, delta[t+1]) 

            count+=1
            ite+=1
        epoch+=1 #Number of epochs
        cost_final = cost_total(X_valid, theta, valid_class_enc, m)
        print("Cost on val dataset after {} epochs is = {}".format(epoch, cost_final))
        if (abs(cost_final-cost_init) < 1e-07):
            print("cost initial= {} , cost final={} , change in cost= {}".format(cost_init,cost_final, cost_final-cost_init))
            break
        cost_init = cost_final
    epochs.append(epoch)
    train_time.append(time.time()-start)
    train_accuracy.append(calc_accuracy(X_train, theta, train_class_enc))
    valid_accuracy.append(calc_accuracy(X_valid, theta, valid_class_enc))
    test_accuracy.append(calc_accuracy(X_test, theta, test_actual_class_enc))
    print("\n------------------------------------------------------------------------------")
    print("The stats for number of units in the hidden layer = {} are as below:".format(arch_test[i]))
    print("------------------------------------------------------------------------------")
    print("The number of epochs = {:2.3f}".format(epochs[-1]))
    print("The training time = {:2.3f}sec".format(train_time[-1]))
    print("The training accuracy is = {:2.3f}%".format(train_accuracy[-1]))
    print("The validation accuracy is = {:2.3f}%".format(valid_accuracy[-1]))
    print("The test accuracy is = {:2.3f}%".format(test_accuracy[-1]))
    print("------------------------------------------------------------------------------\n")

Initial Cost on Val dataset for this epoch 0 = 0.953644483890949
Error on this batch = 5.476332959477003
Error on this batch = 0.5648575713283117
Cost on val dataset after 1 epochs is = 0.09232221590921592
Initial Cost on Val dataset for this epoch 1 = 0.09232221590921592
Error on this batch = 0.5174042573259964
Error on this batch = 0.504727241302468
Cost on val dataset after 2 epochs is = 0.08870852540144292
Initial Cost on Val dataset for this epoch 2 = 0.08870852540144292
Error on this batch = 0.4981373514066121
Error on this batch = 0.49667396036547784
Cost on val dataset after 3 epochs is = 0.08767510648471674
Initial Cost on Val dataset for this epoch 3 = 0.08767510648471674
Error on this batch = 0.49310543664296885
Error on this batch = 0.493645178353701
Cost on val dataset after 4 epochs is = 0.08716716424869865
Initial Cost on Val dataset for this epoch 4 = 0.08716716424869865
Error on this batch = 0.49086207417581296
Error on this batch = 0.492036653194507
Cost on val datase

Error on this batch = 0.4476439569116982
Cost on val dataset after 42 epochs is = 0.07928985142406389
Initial Cost on Val dataset for this epoch 42 = 0.07928985142406389
Error on this batch = 0.4487522205238856
Error on this batch = 0.4455231081123978
Cost on val dataset after 43 epochs is = 0.07896354036984611
Initial Cost on Val dataset for this epoch 43 = 0.07896354036984611
Error on this batch = 0.4468455768664616
Error on this batch = 0.44333152671829523
Cost on val dataset after 44 epochs is = 0.0786279228359681
Initial Cost on Val dataset for this epoch 44 = 0.0786279228359681
Error on this batch = 0.4448896560780375
Error on this batch = 0.4410606378801188
Cost on val dataset after 45 epochs is = 0.0782824241856165
Initial Cost on Val dataset for this epoch 45 = 0.0782824241856165
Error on this batch = 0.44287942587654144
Error on this batch = 0.43870553339794577
Cost on val dataset after 46 epochs is = 0.07792670041698899
Initial Cost on Val dataset for this epoch 46 = 0.07792

Error on this batch = 0.3587361845097709
Cost on val dataset after 83 epochs is = 0.06580692094536644
Initial Cost on Val dataset for this epoch 83 = 0.06580692094536644
Error on this batch = 0.3753124465763737
Error on this batch = 0.35710339534082053
Cost on val dataset after 84 epochs is = 0.06558229081235796
Initial Cost on Val dataset for this epoch 84 = 0.06558229081235796
Error on this batch = 0.37390955057502284
Error on this batch = 0.3554803046130084
Cost on val dataset after 85 epochs is = 0.0653605841910935
Initial Cost on Val dataset for this epoch 85 = 0.0653605841910935
Error on this batch = 0.37251356498744265
Error on this batch = 0.35386474682737346
Cost on val dataset after 86 epochs is = 0.06514166464425183
Initial Cost on Val dataset for this epoch 86 = 0.06514166464425183
Error on this batch = 0.37112126914390403
Error on this batch = 0.35225469145657845
Cost on val dataset after 87 epochs is = 0.06492541354494732
Initial Cost on Val dataset for this epoch 87 = 0.

Error on this batch = 0.29675551361509306
Cost on val dataset after 123 epochs is = 0.05853524696566875
Initial Cost on Val dataset for this epoch 123 = 0.05853524696566875
Error on this batch = 0.32525041317424186
Error on this batch = 0.2954346094209867
Cost on val dataset after 124 epochs is = 0.05838831184308377
Initial Cost on Val dataset for this epoch 124 = 0.05838831184308377
Error on this batch = 0.3243542717386287
Error on this batch = 0.29413528907028386
Cost on val dataset after 125 epochs is = 0.05824237710327827
Initial Cost on Val dataset for this epoch 125 = 0.05824237710327827
Error on this batch = 0.3234747837965617
Error on this batch = 0.29285816958073446
Cost on val dataset after 126 epochs is = 0.05809739176594371
Initial Cost on Val dataset for this epoch 126 = 0.05809739176594371
Error on this batch = 0.32261117093566255
Error on this batch = 0.29160377313731195
Cost on val dataset after 127 epochs is = 0.05795330855226294
Initial Cost on Val dataset for this ep

Cost on val dataset after 163 epochs is = 0.05312195065512574
Initial Cost on Val dataset for this epoch 163 = 0.05312195065512574
Error on this batch = 0.2945222568266474
Error on this batch = 0.2578463966693603
Cost on val dataset after 164 epochs is = 0.052989984293944276
Initial Cost on Val dataset for this epoch 164 = 0.052989984293944276
Error on this batch = 0.29364220968118726
Error on this batch = 0.25709931223190724
Cost on val dataset after 165 epochs is = 0.05285747587412597
Initial Cost on Val dataset for this epoch 165 = 0.05285747587412597
Error on this batch = 0.29274236856745345
Error on this batch = 0.25634981833683856
Cost on val dataset after 166 epochs is = 0.0527243460435061
Initial Cost on Val dataset for this epoch 166 = 0.0527243460435061
Error on this batch = 0.29182240912999263
Error on this batch = 0.25559659818069674
Cost on val dataset after 167 epochs is = 0.05259051691895205
Initial Cost on Val dataset for this epoch 167 = 0.05259051691895205
Error on th

...

Error on this batch = 0.06698322588156466
Cost on val dataset after 2653 epochs is = 0.024600229151709598
Initial Cost on Val dataset for this epoch 2653 = 0.024600229151709598
Error on this batch = 0.07719959600008505
Error on this batch = 0.06697388604642662
Cost on val dataset after 2654 epochs is = 0.024599995630666664
Initial Cost on Val dataset for this epoch 2654 = 0.024599995630666664
Error on this batch = 0.07718493812387153
Error on this batch = 0.06696456162302483
Cost on val dataset after 2655 epochs is = 0.024599763014485493
Initial Cost on Val dataset for this epoch 2655 = 0.024599763014485493
Error on this batch = 0.07717029128997994
Error on this batch = 0.0669552524826535
Cost on val dataset after 2656 epochs is = 0.024599531302697104
Initial Cost on Val dataset for this epoch 2656 = 0.024599531302697104
Error on this batch = 0.07715565546380462
Error on this batch = 0.06694595849801015
Cost on val dataset after 2657 epochs is = 0.024599300494835826
Initial Cost on Val

Error on this batch = 0.06662050418427701
Cost on val dataset after 2693 epochs is = 0.024591589501002667
Initial Cost on Val dataset for this epoch 2693 = 0.024591589501002667
Error on this batch = 0.07662150410443358
Error on this batch = 0.06661169345437737
Cost on val dataset after 2694 epochs is = 0.024591391803072224
Initial Cost on Val dataset for this epoch 2694 = 0.024591391803072224
Error on this batch = 0.07660725601131649
Error on this batch = 0.06660289399303021
Cost on val dataset after 2695 epochs is = 0.024591194989576334
Initial Cost on Val dataset for this epoch 2695 = 0.024591194989576334
Error on this batch = 0.07659301722061161
Error on this batch = 0.06659410572075491
Cost on val dataset after 2696 epochs is = 0.02459099905982926
Initial Cost on Val dataset for this epoch 2696 = 0.02459099905982926
Error on this batch = 0.07657878768242996
Error on this batch = 0.06658532855913843
Cost on val dataset after 2697 epochs is = 0.02459080401312949
Initial Cost on Val d

Cost on val dataset after 2733 epochs is = 0.024584362905995163
Initial Cost on Val dataset for this epoch 2733 = 0.024584362905995163
Error on this batch = 0.07605832727890087
Error on this batch = 0.06626776053837743
Cost on val dataset after 2734 epochs is = 0.024584199851042075
Initial Cost on Val dataset for this epoch 2734 = 0.024584199851042075
Error on this batch = 0.0760444106934275
Error on this batch = 0.06625935738627083
Cost on val dataset after 2735 epochs is = 0.024584037636212527
Initial Cost on Val dataset for this epoch 2735 = 0.024584037636212527
Error on this batch = 0.0760305012900488
Error on this batch = 0.06625096305315906
Cost on val dataset after 2736 epochs is = 0.024583876259850392
Initial Cost on Val dataset for this epoch 2736 = 0.024583876259850392
Error on this batch = 0.07601659901315304
Error on this batch = 0.06624257749833697
Cost on val dataset after 2737 epochs is = 0.024583715720268927
Initial Cost on Val dataset for this epoch 2737 = 0.0245837157

Error on this batch = 0.06594626444454997
Cost on val dataset after 2773 epochs is = 0.024578476800931483
Initial Cost on Val dataset for this epoch 2773 = 0.024578476800931483
Error on this batch = 0.0755067086153139
Error on this batch = 0.06593818204943494
Cost on val dataset after 2774 epochs is = 0.024578345752027
Initial Cost on Val dataset for this epoch 2774 = 0.024578345752027
Error on this batch = 0.075493035142022
Error on this batch = 0.06593010743364984
Cost on val dataset after 2775 epochs is = 0.024578215452832588
Initial Cost on Val dataset for this epoch 2775 = 0.024578215452832588
Error on this batch = 0.07547936657133036
Error on this batch = 0.06592204058333294
Cost on val dataset after 2776 epochs is = 0.024578085900472047
Initial Cost on Val dataset for this epoch 2776 = 0.024578085900472047
Error on this batch = 0.07546570284572975
Error on this batch = 0.06591398148514603
Cost on val dataset after 2777 epochs is = 0.024577957092041434
Initial Cost on Val dataset

Error on this batch = 0.06564451925099067
Cost on val dataset after 2811 epochs is = 0.024573997823053084
Initial Cost on Val dataset for this epoch 2811 = 0.024573997823053084
Error on this batch = 0.07499008488993421
Error on this batch = 0.0656367264421734
Cost on val dataset after 2812 epochs is = 0.02457389302923282
Initial Cost on Val dataset for this epoch 2812 = 0.02457389302923282
Error on this batch = 0.07497655821789019
Error on this batch = 0.06562894117400507
Cost on val dataset after 2813 epochs is = 0.02457378885896951
Initial Cost on Val dataset for this epoch 2813 = 0.02457378885896951
Error on this batch = 0.07496303436210433
Error on this batch = 0.06562116344588254
Cost on val dataset after 2814 epochs is = 0.024573685308570355
Initial Cost on Val dataset for this epoch 2814 = 0.024573685308570355
Error on this batch = 0.07494951327292534
Error on this batch = 0.06561339325737496
Cost on val dataset after 2815 epochs is = 0.02457358237432962
Initial Cost on Val data

In [48]:
calc_accuracy(X_test, theta, test_actual_class_enc)

5.6000000000000005

In [44]:
train_accuracy

[94.95927601809954, 93.78280542986425, 92.85972850678733]